**Import statements**


In [1]:
import pandas as pd
from datetime import datetime
from datetime import timedelta
import numpy as np
import pykrige.kriging_tools as kt
from pykrige.ok import OrdinaryKriging
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from sklearn.linear_model import PassiveAggressiveRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error

import constants
import functions

%matplotlib inline

**Load Data**

In [2]:
all_static_data = functions.load_static_sensors_calibrated()
all_static_data = functions.transform_to_grid_coordinates(all_static_data)
mobile_sensor_data = functions.load_mobile_sensors()
mobile_sensor_data = functions.transform_to_grid_coordinates(mobile_sensor_data)

**Compute predictions for stationary sensors**

In [3]:
# Hyperparameter search definition

cs=[0.0000002, 0.00002, 0.0002, 0.002, 0.02, 0.2, 2]
epsilons=[0.001,0.1, 0.5, 1, 2.5, 5]
losses=['epsilon_insensitive']
#losses=['epsilon_insensitive','squared_epsilon_insensitive']
include_temp_hum = [False, True]
input_window_sizes = range(8,50, 8)

In [4]:
def test_on_data(c, epsilon, loss, temp_hum, window_size, all_static_data):
    mae = []
    mse = []
    squares = []
    pred=[]

    start_time = '2018-07-03 00:00:00'
    start_window = '2018-07-05 12:00:00'
    end_window = '2018-07-05 12:15:00'
    offset_timeint = 1

    number_of_windows = 100
    window = 15
    par_grid = functions.create_par(c, epsilon, loss)
    timeint_on_first_window = 0

    for i in range(1, number_of_windows+1):
        end_time = start_window
        # treinar sem dados moveis
        # testar em dados fixos em vez de móveis
        timeint, par_grid = functions.train(all_static_data, None, start_time, end_time, par_grid, window, timeint_on_first_window, temp_hum=temp_hum)
        to_test = functions.test_mobile(par_grid, timeint, all_static_data, start_window, end_window, 1)
        squares.append(to_test.shape[0])
        mae.append(mean_absolute_error(to_test['PM2.5'], to_test['pred_PM2.5']))
        mse.append(mean_squared_error(to_test['PM2.5'], to_test['pred_PM2.5']))
        pred.append(to_test['pred_PM2.5'].values[0])

        start_time = end_time
        start_window = (datetime.strptime(start_window, '%Y-%m-%d %H:%M:%S') + timedelta(minutes=15)).strftime("%Y-%m-%d %H:%M:%S")
        end_window = (datetime.strptime(end_window, '%Y-%m-%d %H:%M:%S') + timedelta(minutes=15)).strftime("%Y-%m-%d %H:%M:%S")
        timeint_on_first_window = timeint+1
    return np.mean(mae), np.mean(mse)

In [5]:
results = pd.DataFrame(columns=["C", "epsilon", "loss", "temp_hum", "window_size", "mae", "mse"])
for c in cs:
    for epsilon in epsilons:
        for loss in losses:
            # for temp_hum in include_temp_hum:
                # for window_size in input_window_sizes:
                    temp_hum=False
                    window_size=0
                    mae, mse = test_on_data(c, epsilon, loss, temp_hum, window_size, all_static_data)
                    results = results.append({'C': c,
                                             'epsilon': epsilon,
                                             'loss': loss,
                                             'temp_hum': temp_hum,
                                             'window_size': window_size,
                                             'mae': mae,
                                             'mse': mse}, ignore_index=True)
                    print(mae)

0.3032479790813571
0.30874006051233643
0.44762902543948146
0.6157271334221328
1.023627802890817
1.023627802890817
0.10776713789126383
0.1309678826268808
0.3324362217572864
0.5431111923417778
1.023627802890817
1.023627802890817
0.10776707646875129
0.13096568850878923
0.332426555740341
0.5429160513102175
1.023627802890817
1.023627802890817
0.10776679297949432
0.13095546573733616
0.3323806329682857
0.5421502851760718
1.023627802890817
1.023627802890817
0.10776565801652907
0.1309145032097039
0.33220823399056437
0.539568435844983
1.023627802890817
1.023627802890817
0.10776154023040363
0.1307715666932446
0.3314797643786022
0.5327354852180161
1.023627802890817
1.023627802890817
0.10775289849205968
0.1304910549502287
0.33013110911423177
0.5327354852180161
1.023627802890817
1.023627802890817


In [6]:
results.to_csv('results/hyperparameter_search.csv', index=False)
results

,C,epsilon,loss,temp_hum,window_size,mae,mse
0,2.000000e-07,0.001,epsilon_insensitive,False,0,0.303248,0.118995
1,2.000000e-07,0.100,epsilon_insensitive,False,0,0.308740,0.126557
2,2.000000e-07,0.500,epsilon_insensitive,False,0,0.447629,0.278102
3,2.000000e-07,1.000,epsilon_insensitive,False,0,0.615727,0.592117
4,2.000000e-07,2.500,epsilon_insensitive,False,0,1.023628,1.211358
5,2.000000e-07,5.000,epsilon_insensitive,False,0,1.023628,1.211358
6,2.000000e-05,0.001,epsilon_insensitive,False,0,0.107767,0.020651
7,2.000000e-05,0.100,epsilon_insensitive,False,0,0.130968,0.025949
8,2.000000e-05,0.500,epsilon_insensitive,False,0,0.332436,0.146091
9,2.000000e-05,1.000,epsilon_insensitive,False,0,0.543111,0.411811
